### Développement du "Modèle avancé BERT"
Ce notebook implémente un modèle de classification de sentiments utilisant BERT. 
Nous allons fine-tuner un modèle pré-entraîné sur notre jeu de données, puis évaluer ses performances et enregistrer les résultats avec MLFlow.


In [53]:
import pandas as pd

# Charger les données
data_path = "../data/training.1600000.processed.noemoticon.csv"
data = pd.read_csv(data_path, encoding='latin-1', header=None)

In [54]:
# Afficher un échantillon des données
data.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
data_sample = data[['target', 'text']].sample(5)
display(data_sample)

,target,text
538545,0,@Lilayy WHAT !?!?!? There's a picture of Miley...
1286566,4,I'm so Taylor Swift and Katy Perry. Love them.
1056735,4,"@adisti thanks for today, babe! :*"
947654,4,Rumor has it that Girls Gone Wild is filming i...
1148088,4,@quintosential all of your ontd tags are WIN


In [55]:
# Transformer les valeurs de 'target' : 0 reste 0 (négatif) et 4 devient 1 (positif)
data['target'] = data['target'].apply(lambda x: 1 if x == 4 else 0)

# Vérifier un échantillon après la transformation
data_sample = data[['target', 'text']].sample(5)
display(data_sample)


,target,text
308563,0,We just got in from our midnight dig walk. So ...
708522,0,@GabeAlvarez that's EXACTLY what I said this m...
1553750,1,Gettin Ready Gunna Chill 4 A Lido
603440,0,"@hayles All the info is missing tho, well on m..."
521237,0,Hot n sweaty on turnpike lane http://twitpic....


In [56]:
!pip install transformers

In [57]:
import pandas as pd
from transformers import BertTokenizer

# Définir la taille de l'échantillon
sample_size = 1000

# Limiter le dataset à un échantillon stratifié de 10000 lignes basé sur la colonne 'target'
sample_data = data.groupby('target', group_keys=False).apply(lambda x: x.sample(int(sample_size * len(x) / len(data)), random_state=42))

# Vérifier la distribution des classes dans l'échantillon par rapport au dataset d'origine
print("Distribution des classes dans le dataset d'origine :")
print(data['target'].value_counts(normalize=True))
print("\nDistribution des classes dans l'échantillon :")
print(sample_data['target'].value_counts(normalize=True))

# Prétraitement des données textuelles sur cet échantillon
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_text(text):
    tokens = tokenizer(text, max_length=128, truncation=True, padding='max_length', return_tensors='tf')
    return tokens

# Appliquer la fonction de prétraitement
sample_data['tokens'] = sample_data['text'].apply(preprocess_text)

# Afficher un échantillon des données après tokenisation
sample_data_sample = sample_data[['target', 'tokens']].sample(5)
display(sample_data_sample)


C:\Users\pat\AppData\Local\Temp\ipykernel_22424\3146143680.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_data = data.groupby('target', group_keys=False).apply(lambda x: x.sample(int(sample_size * len(x) / len(data)), random_state=42))


Distribution des classes dans le dataset d'origine :
target
0    0.5
1    0.5
Name: proportion, dtype: float64

Distribution des classes dans l'échantillon :
target
0    0.5
1    0.5
Name: proportion, dtype: float64


,target,tokens
1082625,1,"[input_ids, token_type_ids, attention_mask]"
1416899,1,"[input_ids, token_type_ids, attention_mask]"
1130118,1,"[input_ids, token_type_ids, attention_mask]"
480285,0,"[input_ids, token_type_ids, attention_mask]"
421299,0,"[input_ids, token_type_ids, attention_mask]"


### Charger et préparer le modèle BERT :

In [58]:
# Chargement et préparation du modèle BERT pré-entraîné
from transformers import TFBertForSequenceClassification
import tensorflow as tf

# Charger le modèle BERT pré-entraîné
model_bert = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Compilations du modèle
model_bert.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
                   loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                   metrics=['accuracy'])


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Préparer les données et entraîner le modèle 

In [59]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

# Extract necessary elements for BERT (input_ids and attention_mask)
input_ids = np.array([t['input_ids'].numpy()[0] for t in sample_data['tokens']])
attention_masks = np.array([t['attention_mask'].numpy()[0] for t in sample_data['tokens']])
y = np.array(sample_data['target'].values)

# Split the data using train_test_split
X_train_ids, X_test_ids, X_train_mask, X_test_mask, y_train, y_test = train_test_split(
    input_ids, attention_masks, y, test_size=0.2, random_state=42
)

# Convert back to TensorFlow tensors
X_train_ids = tf.convert_to_tensor(X_train_ids)
X_test_ids = tf.convert_to_tensor(X_test_ids)
X_train_mask = tf.convert_to_tensor(X_train_mask)
X_test_mask = tf.convert_to_tensor(X_test_mask)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

# Prepare TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": X_train_ids, "attention_mask": X_train_mask}, y_train)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": X_test_ids, "attention_mask": X_test_mask}, y_test)).batch(32)

# Train the model
history = model_bert.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=3
)


Epoch 1/3


25/25 [==============================] - 305s 11s/step - loss: 0.6879 - accuracy: 0.5587 - val_loss: 0.6320 - val_accuracy: 0.6350
Epoch 2/3
25/25 [==============================] - 276s 11s/step - loss: 0.5402 - accuracy: 0.7375 - val_loss: 0.5430 - val_accuracy: 0.7350
Epoch 3/3
25/25 [==============================] - 297s 12s/step - loss: 0.3273 - accuracy: 0.8763 - val_loss: 0.5050 - val_accuracy: 0.7750


In [61]:
# Évaluation des performances
from sklearn.metrics import classification_report

# Préparer les données de test dans le bon format
test_inputs = {"input_ids": X_test_ids, "attention_mask": X_test_mask}

# Prédictions
y_pred = model_bert.predict(test_inputs)
y_pred_labels = tf.argmax(y_pred.logits, axis=1).numpy()

# Afficher le rapport de classification
print(classification_report(y_test, y_pred_labels))


7/7 [==============================] - 22s 3s/step
              precision    recall  f1-score   support

           0       0.75      0.80      0.77        96
           1       0.80      0.75      0.78       104

    accuracy                           0.78       200
   macro avg       0.78      0.78      0.77       200
weighted avg       0.78      0.78      0.78       200



D'après le rapport de classification, voici les observations :

### Précision, rappel et F1-score par classe :
- **Classe 0 (Négatif)** :
  - Précision : 0.75
  - Rappel : 0.80
  - F1-score : 0.77
  - Le modèle détecte correctement 75 % des exemples négatifs qu'il prédit, et identifie 80 % des véritables exemples négatifs.

- **Classe 1 (Positif)** :
  - Précision : 0.80
  - Rappel : 0.75
  - F1-score : 0.78
  - Le modèle est plus précis dans ses prédictions pour cette classe, mais il manque 25 % des exemples positifs.

### Moyennes globales :
- **Accuracy (Exactitude)** : 0.78, indiquant que 78 % des prédictions totales sont correctes.
- **Macro avg** : Les moyennes arithmétiques des scores (précision, rappel, F1-score) pour les deux classes sont équilibrées, toutes autour de 0.78.
- **Weighted avg** : Les moyennes pondérées, tenant compte de la proportion de chaque classe, montrent également une performance cohérente à 0.78.

### Interprétation :
Le modèle montre une performance correcte, mais avec des marges d'amélioration possibles. Un F1-score de 0.78 reste acceptable, mais un ajustement des hyperparamètres (comme le taux d'apprentissage ou le nombre d'époques) pourrait optimiser les résultats. Vérifier s'il y a un déséquilibre entre les classes serait également pertinent pour appliquer des techniques de rééquilibrage.

### Recommandations pour l'amélioration :
- Augmenter le nombre d'époques pour voir si les performances s'améliorent sans sur-ajuster le modèle.
- Utiliser des techniques de **data augmentation** pour renforcer la généralisation, surtout si la taille du dataset est limitée.
- Expérimenter avec des variantes de BERT ou ajuster plus finement certaines couches pour cette tâche spécifique pourrait aussi offrir des améliorations.

In [62]:
# Enregistrement des expérimentations et des résultats avec MLFlow
import mlflow
import mlflow.keras
import os

# Chemin relatif pour le dossier "models"
relative_models_path = os.path.join("..", "models")

# Configuration du chemin pour stocker les artefacts de MLFlow
mlflow.set_tracking_uri(relative_models_path)

# Créer une nouvelle expérience ou utiliser une existante
experiment_name = "BERT_Embedding_Experiment"
mlflow.set_experiment(experiment_name)

# Démarrer une nouvelle session MLFlow pour BERT
with mlflow.start_run():
    mlflow.log_param("model", "BERT")
    mlflow.log_param("epochs", 3)
    mlflow.log_metric("accuracy", history.history['accuracy'][-1])
    mlflow.log_metric("val_accuracy", history.history['val_accuracy'][-1])
    
    # Enregistrer le modèle BERT comme artefact
    mlflow.keras.log_model(model_bert, "model_bert")

print(f"Modèle BERT enregistré dans {relative_models_path}.")

2024/10/20 11:04:31 INFO mlflow.tracking.fluent: Experiment with name 'BERT_Embedding_Experiment' does not exist. Creating a new experiment.
2024/10/20 11:04:31 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:


Modèle BERT enregistré dans ..\models.



## Conclusion
Le modèle BERT a été fine-tuné sur notre jeu de données pour l'analyse de sentiments. 
Les résultats ont été enregistrés avec MLFlow, et le modèle a montré des performances prometteuses en termes de précision.
